In [1]:
from ftplib import FTP
import datetime as dt
from tqdm import tqdm
import pandas as pd

In [2]:
ROOT_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline'
FTP_CONF = {
    'host' : 'calc1.cor-e.fr',
    'port' : 21,
    'user' : 'verso_energy',
    'passwd' : '@VeRsoEnerGy2023!'
}

In [3]:
def kpler_ftp():
    ftp = FTP()
    ftp.connect(host=FTP_CONF['host'], port=FTP_CONF['port'])
    ftp.login(user=FTP_CONF['user'], passwd=FTP_CONF['passwd'])
    return ftp

In [4]:
def get_electricity_futures(date : dt.date, country):
    ftp = kpler_ftp()
    date_str = date.strftime(format='%Y-%m-%d')
    year, month, date = date_str.split('-')
    for e in ['Base', 'Peak']:
        with open(f'{ROOT_PATH}/process/data/KPLER/Futures/Elec/EEX/{country}/{date_str.replace('-', '')}_Futures_Elec_EEX_{country}_{e}.csv', 'wb') as local_file:
            try:
                ftp.retrbinary(f'RETR 2_Price/Actual/Futures/Electricity/EEX/{country}/{year}/{month}/CORE_PRICE_ACTUAL_Futures_Electricity_EEX_{country}_{e}_Daily_{date_str.replace('-', '')}.csv', 
                            local_file.write)
            except:
                with open(f'{ROOT_PATH}/log.txt', 'a') as log_file:
                    log_file.write(f'{date_str} - {e}\n')

In [5]:
def get_DA_gas_price(date : dt.date, country):
    ftp = kpler_ftp()
    date_str = date.strftime(format='%Y-%m-%d')
    year, month, date = date_str.split('-')
    with open(f'{ROOT_PATH}/process/data/KPLER/DayAhead/Gas/EEX/{country}/{date_str.replace('-', '')}_DayAhead_Gas_EEX_FR.csv', 'wb') as local_file:
        try:
            ftp.retrbinary(f'RETR 2_Price/Actual/Day_Ahead/EEX/Gas/{country}/{year}/{month}/CORE_PRICE_ACTUAL_DayAhead_Gas_Midday_EEX_{country}_{date_str.replace('-', '')}.csv', 
                            local_file.write)
        except Exception as e:
            with open(f'{ROOT_PATH}/log_gas.txt', 'a') as log_file:
                log_file.write(f'{date_str} - {e}\n')

In [7]:
for date in tqdm(pd.date_range(start='2024-12-16', end='2024-12-18', freq='D', inclusive='both').date):
    # if not((date.isoweekday() == 6) or (date.isoweekday() == 7)):
    #     get_electricity_futures(date, 'FR')
    get_DA_gas_price(date, 'FR')

100%|██████████| 3/3 [00:01<00:00,  2.29it/s]
